In [1]:
import pandas as pd
from splinter import Browser
# from bs4 import BeautifulSoup as bs
import time

In [2]:
url = "https://gis.cdc.gov/grasp/PSA/MapView.html"

In [3]:
def init_browser():
    executable_path = {"executable_path": "../../chromedriver.exe"}
#     later change headless to true to see if it helps with errors I think are load time related
    return Browser("chrome", **executable_path, headless=False)

In [4]:
browser = init_browser()
browser.visit(url)

In [5]:
# pause here to let browser catch up, could add time.sleep(5)

# Gets by the initial disclaimer popup message
proceed = browser.find_by_id("btnModalAccept")
proceed.click()

In [6]:
browser.find_by_text("State Summaries").click()

In [7]:
StateDropdownBtn = browser.find_by_id("btnDDLState").click()

states_dropdown = browser.find_by_id("ulStateList").value

states = states_dropdown.splitlines()

states_to_drop = [
    'Regions', 'East North Central', 'East South Central', 'Mid Atlantic', 
    'Mountain', 'New England', 'Pacific', 'South Atlantic', 'West North Central', 
    'West South Central', 'States', 'National', '- Select state or Region -'
]

for i in states_to_drop:
    if i in states:
        states.remove(i)

In [8]:
EventTypeDropdownBtn = browser.find_by_id("btnDDLEventType").click()

event_dropdown = browser.find_by_id("ulEventType").value

event_type = event_dropdown.splitlines()

event_type.remove('All HAIs')

In [9]:
YearsDropdownBtn = browser.find_by_id("btnDDLYears").click()

years_dropdown = browser.find_by_id("ulYear").value

years = years_dropdown.splitlines()

years.remove('All Years')

In [ ]:
scrape_df = pd.DataFrame()

for st in states:
    browser.find_by_id("btnDDLState").click()
    browser.find_by_text(st).click()
    
    for ev in event_type:
        browser.find_by_id("btnDDLEventType").click()
        browser.find_by_text(ev).click()
        
        for yr in years:
            browser.find_by_id("btnDDLYears").click()
            browser.find_by_text(yr).click()
            
#           1 second works most of the time but bumped it up to avoid fatal stale element reference error
            time.sleep(3)
            
            tds = browser.find_by_tag("td")
            i = 0;
            
            while i < len(tds):
                row = []
                j = 0
                while j < 7:
                    if tds[i].value != "Gram Positive" and tds[i].value != "Gram Negative" and tds[i].value:
                        row.append(tds[i].value)
                        j = j + 1
                        i = i + 1 
                    else:
                        i = i + 1      
                row.append(ev)
                row.append(yr)
                scrape_df = scrape_df.append([row], ignore_index=True)
                print(row)    
                

In [20]:
scrape_df.to_csv('CDC_AR_scrape.csv', index=False)

In [54]:
browser.quit()